Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 4

## Assignment

- [ ] Watch Aaron Gallant's [video #1](https://www.youtube.com/watch?v=pREaWFli-5I) (12 minutes) & [video #2](https://www.youtube.com/watch?v=bDQgVt4hFgY) (9 minutes) to learn about the mathematics of Logistic Regression.
- [ ] Do train/validate/test split with the Tanzania Waterpumps data.
- [ ] Do one-hot encoding. (Remember it may not work with high cardinality categoricals.)
- [ ] Use scikit-learn for logistic regression.
- [ ] Get your validation accuracy score.
- [ ] Get and plot your coefficients.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

> [Do Not Copy-Paste.](https://docs.google.com/document/d/1ubOw9B3Hfip27hF2ZFnW3a3z9xAgrUDRReOEo-FHCVs/edit) You must type each of these exercises in, manually. If you copy and paste, you might as well not even do them. The point of these exercises is to train your hands, your brain, and your mind in how to read, write, and see code. If you copy-paste, you are cheating yourself out of the effectiveness of the lessons.


## Stretch Goals

### Doing
- [ ] Add your own stretch goal(s) !
- [ ] Clean the data. For ideas, refer to [The Quartz guide to bad data](https://github.com/Quartz/bad-data-guide),  a "reference to problems seen in real-world data along with suggestions on how to resolve them." One of the issues is ["Zeros replace missing values."](https://github.com/Quartz/bad-data-guide#zeros-replace-missing-values)
- [ ] Make exploratory visualizations.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).


#### Exploratory visualizations

Visualize the relationships between feature(s) and target. I recommend you do this with your training set, after splitting your data. 

For this problem, you may want to create a new column to represent the target as a number, 0 or 1. For example:

```python
train['functional'] = (train['status_group']=='functional').astype(int)
```



You can try [Seaborn "Categorical estimate" plots](https://seaborn.pydata.org/tutorial/categorical.html) for features with reasonably few unique values. (With too many unique values, the plot is unreadable.)

- Categorical features. (If there are too many unique values, you can replace less frequent values with "OTHER.")
- Numeric features. (If there are too many unique values, you can [bin with pandas cut / qcut functions](https://pandas.pydata.org/pandas-docs/stable/getting_started/basics.html?highlight=qcut#discretization-and-quantiling).)

You can try [Seaborn linear model plots](https://seaborn.pydata.org/tutorial/regression.html) with numeric features. For this problem, you may want to use the parameter `logistic=True`

You do _not_ need to use Seaborn, but it's nice because it includes confidence intervals to visualize uncertainty.

#### High-cardinality categoricals

This code from the previous assignment demonstrates how to replace less frequent values with 'OTHER'

```python
# Reduce cardinality for NEIGHBORHOOD feature ...

# Get a list of the top 10 neighborhoods
top10 = train['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10,
# replace the neighborhood with 'OTHER'
train.loc[~train['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
test.loc[~test['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'
```

#### Pipelines

[Scikit-Learn User Guide](https://scikit-learn.org/stable/modules/compose.html) explains why pipelines are useful, and demonstrates how to use them:

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:
> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

### Reading
- [ ] [How (and why) to create a good validation set](https://www.fast.ai/2017/11/13/validation-sets/)
- [ ] [Always start with a stupid model, no exceptions](https://blog.insightdatascience.com/always-start-with-a-stupid-model-no-exceptions-3a22314b9aaa)
- [ ] [Statistical Modeling: The Two Cultures](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).



In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module4')

In [2]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [3]:
import pandas as pd

train_features = pd.read_csv('../data/tanzania/train_features.csv')
train_labels = pd.read_csv('../data/tanzania/train_labels.csv')
test_features = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

assert train_features.shape == (59400, 40)
assert train_labels.shape == (59400, 2)
assert test_features.shape == (14358, 40)
assert sample_submission.shape == (14358, 2)

In [6]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_val, y_train, y_val=train_test_split(train_features,train_labels)

In [9]:
X_train.shape,X_val.shape

((44550, 40), (14850, 40))

In [10]:
y_train.shape, y_val.shape

((44550, 2), (14850, 2))

In [78]:
y_train=y_train['status_group']
y_val=y_val['status_group']

In [80]:
y_train.value_counts(normalize=True)

functional                 0.540516
non functional             0.387160
functional needs repair    0.072323
Name: status_group, dtype: float64

In [15]:
X_train.describe(exclude='number').T.sort_values('unique')

,count,unique,top,freq
recorded_by,44550,1,GeoData Consultants Ltd,44550
public_meeting,42080,2,True,38269
permit,42262,2,True,29140
source_class,44550,3,groundwater,34271
management_group,44550,5,user-group,39407
quantity_group,44550,5,enough,24817
quantity,44550,5,enough,24817
waterpoint_type_group,44550,6,communal standpipe,25998
quality_group,44550,6,good,38126
payment_type,44550,7,never pay,19080


In [23]:
X_train.groupby('region').population.mean().sort_values()

region
Mbeya              0.000000
Dodoma             0.000000
Kagera             0.000000
Tabora             0.000000
Shinyanga         13.903027
Mwanza            66.263476
Iringa            92.400612
Kilimanjaro      105.970437
Ruvuma           195.267930
Dar es Salaam    220.594059
Tanga            256.433923
Morogoro         256.864632
Arusha           260.593886
Mtwara           261.782946
Singida          266.128750
Manyara          309.645868
Pwani            333.136896
Lindi            363.603448
Rukwa            376.278223
Kigoma           502.086833
Mara             534.400137
Name: population, dtype: float64

In [26]:
df=X_train.merge(y_train)
df.groupby('region').status_group.describe()

,count,unique,top,freq
region,,,,
Arusha,2519,3,functional,1724
Dar es Salaam,606,3,functional,333
Dodoma,1637,3,functional,741
Iringa,3919,3,functional,3071
Kagera,2494,3,functional,1283
Kigoma,2119,3,functional,1020
Kilimanjaro,3315,3,functional,1992
Lindi,1160,3,non functional,756
Manyara,1186,3,functional,732


In [16]:
import category_encoders as ce

In [81]:
categorical_features=['region', 'quantity']
numeric_features=X_train.select_dtypes('number').columns.drop('id').tolist()
categorical_features

['region', 'quantity']

In [82]:
#theres a lot of 'numbers' that should be strings
features=categorical_features + numeric_features
features

['region',
 'quantity',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year']

In [83]:
#this method does one hot encoding on all categorical columns. 
#if the faux numbers were encoded as strings they would be able to be one hot encoded as well
X_train_subset=X_train[features]
X_val_subset=X_val[features]

encoder=ce.OneHotEncoder(use_cat_names=True)
X_train_encoded=encoder.fit_transform(X_train_subset)
X_val_encoded=encoder.transform(X_val_subset)

In [84]:
from sklearn.preprocessing import MinMaxScaler
#the scaler puts every value on a scale of 0-1 
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train_encoded)
X_val_scaled=scaler.transform(X_val_encoded)

In [85]:
from sklearn.linear_model import LogisticRegressionCV

In [87]:
model= LogisticRegressionCV(n_jobs=-1)
model.fit(X_train_scaled, y_train)
model.score(X_val_scaled, y_val)

/Users/eyvonnegeordan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Users/eyvonnegeordan/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/Users/eyvonnegeordan/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.6543434343434343

## And Starting Over For Practice

In [96]:
X_train['district_code'].value_counts().shape

(20,)

In [99]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,44550.0,37072.323030,21461.433528,0.00000,18460.500000,37012.500000,55606.750000,7.424700e+04
amount_tsh,44550.0,297.409420,2334.771170,0.00000,0.000000,0.000000,20.000000,2.500000e+05
gps_height,44550.0,668.074658,691.898377,-90.00000,0.000000,371.000000,1320.000000,2.770000e+03
longitude,44550.0,34.059779,6.614160,0.00000,33.083484,34.910318,37.188418,4.034519e+01
latitude,44550.0,-5.697552,2.943403,-11.64944,-8.525173,-5.017653,-3.324285,-2.000000e-08
num_private,44550.0,0.459955,10.802985,0.00000,0.000000,0.000000,0.000000,1.402000e+03
district_code,44550.0,5.639259,9.630654,0.00000,2.000000,3.000000,5.000000,8.000000e+01
population,44550.0,177.919080,466.885536,0.00000,0.000000,25.000000,210.000000,3.050000e+04
construction_year,44550.0,1300.732480,951.552524,0.00000,0.000000,1985.000000,2004.000000,2.013000e+03


In [98]:
X_train['region_code']=X_train['region_code'].astype(str)

/Users/eyvonnegeordan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [100]:
X_train['district_code']=X_train['district_code'].astype(str)

/Users/eyvonnegeordan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [101]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,44550.0,37072.323030,21461.433528,0.00000,18460.500000,37012.500000,55606.750000,7.424700e+04
amount_tsh,44550.0,297.409420,2334.771170,0.00000,0.000000,0.000000,20.000000,2.500000e+05
gps_height,44550.0,668.074658,691.898377,-90.00000,0.000000,371.000000,1320.000000,2.770000e+03
longitude,44550.0,34.059779,6.614160,0.00000,33.083484,34.910318,37.188418,4.034519e+01
latitude,44550.0,-5.697552,2.943403,-11.64944,-8.525173,-5.017653,-3.324285,-2.000000e-08
num_private,44550.0,0.459955,10.802985,0.00000,0.000000,0.000000,0.000000,1.402000e+03
population,44550.0,177.919080,466.885536,0.00000,0.000000,25.000000,210.000000,3.050000e+04
construction_year,44550.0,1300.732480,951.552524,0.00000,0.000000,1985.000000,2004.000000,2.013000e+03


In [105]:
desc=X_train.select_dtypes(exclude='number').describe().T.sort_values('unique')

In [109]:
desc=desc[desc['unique']>1]
desc=desc[desc['unique']<22]

In [112]:
#this is a list of all categorical features which can be one-hot-encoded
cat_feats=desc.index.tolist()

In [119]:
num_feats=X_train.select_dtypes('number').drop('id',axis=1).columns.tolist()
all_feats=num_feats+cat_feats

In [120]:
encoder=ce.OneHotEncoder(use_cat_names=True)
X_train_enc=encoder.fit_transform(X_train[all_feats])
X_val_enc=encoder.transform(X_val[all_feats])